# Projeto 01: Reconhecimento de Dígitos

### João Victor Soares Silva | Mat: 20210027300
### Yhasmim de Souza Tigre | Mat: 20210026966 

## Pré-Processamento

In [1]:
import numpy as np
import random as rd

In [2]:
N = 28 #Número de linhas e colunas das matrizes das imagens
P_MAX = 255 #Número máximo de um pixel

In [10]:
# Aqui é só um exemplo menor de como funcionam as fórmulas da simetria no array.
# Acho bom usar pra estudar pra apresentação, então deixei aqui.

# teste = [
# 1, 2, 3, 4,
# 5, 6, 7, 8,
# 9, 10, 11, 12,
# 13, 14, 15, 16]

# lin = 4

# #Vertical
# print('Vertical')
# for i in range(lin):

#     for j in range(int(lin/2)):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*i) + (lin - j - 1)]} = {teste[(lin*i) + j] + teste[(lin*i) + (lin - j - 1)]}')

# #Horizontal
# print('\nHorizontal')
# for i in range(int(lin/2)):
    
#     for j in range(lin):
#         print(f'{teste[(lin*i) + j]} + {teste[(lin*(lin - i - 1)) + j]} = {teste[(lin*i) + j] + teste[(lin*(lin - i - 1)) + j]}')

In [4]:
def reduzir (nome : str, arq : str) -> None:
    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas
    lista_y = []
    lista_intensidade = []
    lista_simetria_vertical = []
    lista_simetria_horizontal = []
    lista_simetria = []

    for i in range (len(aux)): #Salvando as labels
        lista_y.append (int(aux[i][0]))
    
    aux = np.delete (aux, 0, 1) #Removendo as labels
    
    for i in range (len(aux)): #Calculando as intensidades
        soma = 0

        for j in range (len(aux[0])):
            soma += int(aux[i][j])
        
        lista_intensidade.append(soma/P_MAX)
    
    for imagem in aux: #Simetria vertical
        soma = 0

        for i in range(N):
            for j in range(int(N/2)):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[(N*i) + (N - j - 1)]))

        lista_simetria_vertical.append(soma/P_MAX)

    for imagem in aux: #Simetria horizontal
        soma = 0

        for i in range(int(N/2)):
            for j in range(N):
                soma += abs(int(imagem[(N*i) + j]) - int(imagem[N * (N - i - 1) + j]))

        lista_simetria_horizontal.append(soma/P_MAX)
    
    for i in range (len(aux)): #Somando as simetrias
        lista_simetria.append(lista_simetria_vertical[i] + lista_simetria_horizontal[i])

    #Criando e preenchendo o novo .csv
    dados_redux = open (f'{nome}.csv', 'w')

    dados_redux.write ('label;intensidade;simetria\n')
    for i in range (len(aux)):
        dados_redux.write (f'{lista_y[i]};{lista_intensidade[i]};{lista_simetria[i]}\n')
    
    dados_redux.close()

    return

In [5]:
def leitor_csv (arq : str) -> np.array:
    x = []
    y = []

    aux = np.loadtxt (arq, delimiter = ';', dtype = str)
    aux = np.delete (aux, 0, 0) #Removendo a linha dos nomes das colunas

    for linha in aux:
        x.append([linha[1], linha[2]])
        y.append(linha[0])
    
    return np.array(x), np.array(y)

In [6]:
reduzir(nome = 'train_redu', arq = 'train.csv')
reduzir(nome = 'test_redu', arq = 'test.csv')

## Algoritmos

### Perceptron Learning Algorithm (PLA)

In [7]:
class PLA ():
    def __init__(self, n_int : int = 1000, tam_batch : int = 32) -> None:
        self.n_int = n_int
        self.tam_batch = tam_batch

    def acuracia (self, X : np.array, Y : np.array, w_lista : np.array) -> float:
        soma_PCC = 0

        for i in range (len(X)):
            aux = np.sign (np.sum(np.matmul(w_lista, X[i])))

            if (aux == Y[i]):
                soma_PCC += 1
    
        return (soma_PCC/len(X))
    
    def __construtor_PCI (self, X : np.array, Y : np.array) -> np.array:
        lista_PCI_x = []
        lista_PCI_y = []

        for i in range (len(X)):
            aux = np.sign (np.sum(np.matmul(self.w_lista, X[i])))

            if (aux != Y[i]):
                lista_PCI_x.append(X[i])
                lista_PCI_y.append(Y[i])
        
        return np.array(lista_PCI_x), np.array(lista_PCI_y) 

    def fit (self, X : np.array, Y : np.array) -> None:
        lista_PCI_x = X
        lista_PCI_y = Y
        self.w_lista, w_otimo = np.zeros(X.shape[1]), np.zeros(X.shape[1])

        i = 0
        while (len(lista_PCI_x) > 0 or i < self.n_int):
            ale_index = np.random.randint(0, len(lista_PCI_x)) #index aleatório
            ponto_x = lista_PCI_x[ale_index]
            ponto_y = lista_PCI_y[ale_index]

            aux = np.matmul(ponto_x, ponto_y) #x * y
            w_novo = np.add(self.w_lista, aux)

            if (self.acuracia (X, Y, w_otimo) < self.acuracia (X, Y, w_novo)):
                w_otimo = w_novo

            self.w_lista = w_novo

            lista_PCI_x, lista_PCI_y = self.__construtor_PCI (self, X = X, Y = Y)
            i += 1
        
        self.w_lista = w_otimo

        return
    
    def predict (self, X : np.array) -> np.array:
        predict_y = [np.sign(np.matmul(i, self.w_lista)) for i in X]

        return predict_y

    def get_w (self) -> np.array:
        try:
            return self.w_lista
        
        except:
            print ("Não foi possível recuperar w. Por favor, se certifique de treinar o modelo antes.\n")

    def set_w (self, novo_w : np.array) -> None:
        self.w_lista = novo_w

    

### Regressão Linear

### Regressão Logística

In [8]:
class Reg_Log ():
    def __init__ (self, eta = 0.1, n_int = 1000, tam_batch = 32) -> None:
        self.eta = eta
        self.n_int = n_int
        self.tam_batch = tam_batch
    
    def fit (self, X : np.array, Y : np.array, eps : float = 0.0001) -> None:
        lista_X = X
        lista_y = Y
    
        for i in range (len(X)):
            lista_X[i].insert(0, 1) #O 1 será usado para a multiplicação com o bias posteriormente
        
        n_dim = len(lista_X[0])
        n_elem = len(lista_X)
        w_lista = np.zeros(n_dim, dtype = float)

        #Cálculo dos gradientes pelo processo iterativo
        for i in range (self.n_int):
            vsoma = np.zeros(n_dim, dtype = float)

            if (self.tam_batch) < n_elem:
                batch_X = []
                batch_Y = []
                indices = rd.sample(range(n_elem), self.tam_batch)

                for j in indices:
                    batch_X.append(lista_X[j])
                    batch_Y.append(lista_y[j])
            
            else:
                batch_X = lista_X
                batch_Y = lista_y
            
            for xn, yn in zip(batch_X, batch_Y):
                vsoma += (int(yn) * float(xn)) / 1 + np.exp(np.matmul(np.transpose(yn * w_lista), xn))
            
            grad_t = vsoma / self.tam_batch

            #Testando o minimo
            if (np.linalg.norm(grad_t) <  eps):
                break

            w_lista = w_lista + (self.eta * grad_t)
        
        self.w_lista = w_lista

    def predict (self, X : np.array) -> np.array:
        aux = [1 / (1 + np.exp(- np.dot(self.w_lista, i))) for i in X]
        predict_y = [0 if i <= 0.5 else 1 for i in aux]

        return predict_y

    def get_w (self) -> np.array:
        try:
            return self.w_lista
        
        except:
            print ("Não foi possível recuperar w. Por favor, se certifique de treinar o modelo antes.\n")

    def set_w (self, novo_w : np.array) -> None:
        self.w_lista = novo_w
        
        

## Classificação dos Dígitos 1x5

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Classificação de Dígitos Completo

In [9]:
class UmVSTodos ():
    def __init__(self) -> None:
        pass
        
    def fit (self):
        pass

### Perceptron Learning Algorithm (PLA)

### Regressão Linear

### Regressão Logística

### Relatório de Eficácia

## Comparação dos Classificadores

## Implementações Avançadas

### Weight-Decay para a Regressão Logística

### Definindo a Melhor Ordem